[:de:](index-de.ipynb) [:us:](index.ipynb)

In [1]:
#|include: false

!sqlite3 fin.db .databases .quit

main: /home/git_repos/blog/posts/2022-11-13-repay-vs-invest/fin.db r/w


In [2]:
#|include: false
from IPython.display import Markdown as md
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np

In [3]:
#|include: false

# enable sql magic for Jupyter notebook

# load jupyter extention (requires pip install ipython-sql)
%load_ext sql
# disable autocommit
%config SqlMagic.autocommit=False

In [4]:
#|include: false

%sql sqlite:///fin.db
    
# Test connection
v1 = "Ready to go!"
%sql df_test << SELECT :v1 as "Test"
assert(df_test.DataFrame().values[0]==v1)

 * sqlite:///fin.db
Done.
Returning data to local variable df_test


:::{.callout-warning}
**Disclaimer:** This blog post is not a financial advice! This is a toy example. This blog post is full of unrealistic assumption. All numbers are made up. Reach out to a professional financial advisor you trust, if you need financial advise. And most important, take a spreadsheet and do the math yourself with your numbers and your assumptions.
:::

# A tricky question
Last weekend I had a discussion about repaying a mortgage early or instead investing the money in a non-risky asset. 

:man:: *I just did an extra payment to reduce my mortgage.*

:bust_in_silhouette:: *How much is your interest rate?*

:man:: *2.5%* - again, numbers are made up

:bust_in_silhouette:: *But didn't you consider to invest the money instead. Even if you are risk averse, you would get 2.8% in a fixed deposit?*

:man:: *I thought about it. But it didn't make sense.*

:bust_in_silhouette:: *But why? You would have gained 0.3% per year.*

Who is right in this scenario? The answer is ambiguous.

# Making up some numbers
Before we take it apart, let's make up some numbers:

In [5]:
principal = 100000 # mortgage loan
interest_rate = 0.025 # for the loan
m_payment = 5000 # yearly amount to pay for the mortgage (interest and principal)
reference_rate = 0.028 # interest rate for the non-risky investment
lump_sum = 15000 # money that can be invested or used to pay off the loan
ls_year = 2 # year when the lump sum is invested or used as an extra payment for the loan
appreciation = 0.02 #  value gain of the house per year

Besides, we make some assumption for the ease of interpretation.

- No fees, no taxes.
- The yearly payment for the mortgage stayes the some. 
- As soon as the mortgage is payed off, the yearly payment stays the same and will be invested.
- Interest rates for loan and investment are not changing during the entire time.
- The lump sum can either be used to invest or to make an extra payment to reduce the loan.
- The non-risky investment can't be withdrawn before the house is payed of.
- Interest income is being reinvested (accumulating). 

:::{.callout-important}
These assumptions are unrealistic and incomplete for a real live scenario.
Also, changing the numbers can lead to other conclusions than those described in this blog post.
:::

# The simple case

First, let's have a look at the thoughts of :bust_in_silhouette::


In [6]:
#|echo: false

md(f"""
An extra payment towards the mortgage will reduces the amount I have to pay on interest every year by {interest_rate*100:.2f}%. 
That's a value gain of {interest_rate*100:.2f}% each year.\n
If instead the money is invested in the fixed deposit with an interest rate of {reference_rate*100:.2f}%, 
I had an value gain of {reference_rate*100:.2f}% each year.

```loan interest rate - reference rate = {interest_rate*100:.2f}% - {reference_rate*100:.2f}% = {(interest_rate - reference_rate)*100:.2f}%```

The investment has a yearly gain of {(reference_rate - interest_rate)*100:.2f}% compared to the extra payment.\n
Both payments are compounding, as can be seen in the example:
""")




An extra payment towards the mortgage will reduces the amount I have to pay on interest every year by 2.50%. 
That's a value gain of 2.50% each year.

If instead the money is invested in the fixed deposit with an interest rate of 2.80%, 
I had an value gain of 2.80% each year.

```loan interest rate - reference rate = 2.50% - 2.80% = -0.30%```

The investment has a yearly gain of 0.30% compared to the extra payment.

Both payments are compounding, as can be seen in the example:


In [7]:
#|include: false

%%sql simple_calc <<
with plan(year, payment, zinsen, opp_investment) as
(
    SELECT 0 AS year,
           0 AS payment,
           0 AS zinsen,
           0 AS opp_investment
    UNION
    SELECT p.year + 1 AS year,
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END AS payment,
           p.zinsen * (1+:interest_rate) +
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END AS zinsen,
           p.zinsen * (1+:reference_rate) +
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END AS opp_investment
      FROM plan p
     WHERE p.year < 7
)
SELECT year as year,
       payment as payment,
       max(round(zinsen)-:lump_sum, 0) AS savings_of_mortage_interest, 
       max(round(opp_investment)-:lump_sum, 0) AS gain_from_unrisky_investment,
       abs(max(round(zinsen)-:lump_sum, 0) - 
       max(round(opp_investment)-:lump_sum, 0)) AS diff
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable simple_calc


In [8]:
#|echo: false

simple_calc_df = simple_calc.DataFrame().set_index('year').reset_index(drop = True).rename_axis('Year', axis=1)
simple_calc_df.columns = ['Payment', 'Interest savings through loan reduction', 'Gains from investment', 'Difference']
simple_calc_df

,Payment,Interest savings through loan reduction,Gains from investment,Difference
0,0,0.0,0.0,0.0
1,0,0.0,0.0,0.0
2,15000,0.0,0.0,0.0
3,0,375.0,420.0,45.0
4,0,759.0,805.0,46.0
5,0,1153.0,1201.0,48.0
6,0,1557.0,1606.0,49.0
7,0,1971.0,2021.0,50.0


In [9]:
#|echo: false

md(f"""In this simple example we have a value gain of {simple_calc_df['Difference'].values[-1:][0]} € 
after the fifth year of investing the extra money compared to an extra payment for the loan.""")

In this simple example we have a value gain of 50.0 € 
after the fifth year of investing the extra money compared to an extra payment for the loan.

# Amortization plan
Now let's get familiar with the numbers and draw the entire amortization plan without extra payment or investments.

In [10]:
#|include: false

%%sql amor_plan <<
with plan(year, interest, principal, balance) as
(
    SELECT 0 AS year,
           NULL AS interest,
           NULL AS principal,
           :principal AS balance
    UNION
    SELECT p.year + 1 AS year,
           -p.balance * :interest_rate AS interest,
           MIN(:m_payment - p.balance * :interest_rate, p.balance) AS principal,
           p.balance - (min(:m_payment - p.balance * :interest_rate, p.balance) /*principal*/ ) AS balance
      FROM plan p
     WHERE p.balance - (min(:m_payment - p.balance * :interest_rate, p.balance) + 0) /*Restschuld*/ >= 0 
       AND MIN(:m_payment - p.balance * :interest_rate, p.balance) /*principal*/ > 0 
)
SELECT year,
       interest AS interest, 
       principal AS principal,  
       -balance AS balance
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable amor_plan


In [11]:
#|echo: false
amor_plan_df = amor_plan.DataFrame().set_index('year').reset_index(drop = True).rename_axis('Year', axis=1)
amor_plan_df.loc['Total:'] = amor_plan_df.sum()
amor_plan_df.loc['Total:',['balance']]=amor_plan_df[['balance']].max() 
amor_plan_df.columns = ['Interest', 'Principal', 'Loan Balance']

vmin = -interest_rate*principal
vmax = m_payment-interest_rate*principal

(amor_plan_df.style
            .format('{:.0f}', na_rep="")
            .bar(subset=pd.IndexSlice[amor_plan_df[amor_plan_df['Interest']<=0].index,'Interest'],
                     align='right', vmin=vmin, vmax=0, cmap="autumn", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
                .bar(subset=pd.IndexSlice[amor_plan_df[amor_plan_df['Principal']>=0].index,'Principal'],
                     align='left', vmin=0, vmax=m_payment, cmap="summer_r", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
            #.bar(subset=['interest'], align='right', vmin=vmin, vmax=0, cmap="autumn",
            #     height=80, width=100, props="width: 100px; border-right: 1px solid gray; border-left: 1px solid gray;"
            #    )
            #.bar(subset=['principal'], align='left', vmin=vmin, vmax=m_payment, cmap="summer_r", 
            #     height=80, width=100, props="width: 100px; border-right: 1px solid gray;"
            #    )
            .bar(subset=['Loan Balance'], align='right', vmin=-2.5*principal, vmax=0, cmap="PuRd_r", 
                 height=80, width=80, props="width: 80px; border-right: 1px solid gray;"
                )
                .set_table_styles({
                   ('Total:'): [{'selector': 'th', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'}]
                   }, overwrite=False, axis=1)
               .set_table_styles([{'selector': 'table', 'props': 'border-spacing: 2px'},
                                  {'selector': 'thead', 'props': 'border: 1px solid gray'}, 
                                  {'selector': 'th', 'props': 'text-align: center; padding: 4.5px;'},
                                  {'selector': 'th.col_heading', 'props': 'border: 1px solid gray'},
                                  {'selector': 'tbody', 'props': 'border: 1px solid gray'},
                                  {'selector': 'td', 'props': 'text-align: center; border-left: 1px solid gray; border-right: 1px solid gray;'}
                                 ], overwrite=False)
)

,Interest,Principal,Loan Balance
0,,,-100000
1,-2500,2500,-97500
2,-2438,2562,-94938
3,-2373,2627,-92311
4,-2308,2692,-89619
5,-2240,2760,-86859
6,-2171,2829,-84031
7,-2101,2899,-81131
8,-2028,2972,-78160
9,-1954,3046,-75114


In [12]:
#|echo: false
md(f"With a yearly fixed payment of {m_payment:,.0f} € the mortgage of {principal:,.0f} € will be payed of after {amor_plan_df.index.values[-2]:,.0f} years. "+
      f"The amount of interest paid for the loan is {-amor_plan_df.loc['Total:','Interest']:,.0f} €"
     )

With a yearly fixed payment of 5,000 € the mortgage of 100,000 € will be payed of after 29 years. The amount of interest paid for the loan is 40,359 €

The amortization schedule nicely shows how the interest decreases and principal increases over time.

# Investment payment
Next, we extend the amortization plan with columns for

- Extra payment for the loan
- Payment on investment
- Compounding balance for investment
- Appreciation of the principal payed sofar
- The Net Worth (*asset values - liabilities*)
- The Net Worth including appreciation

In the first scenario we **invest** the lump sum.

In [13]:
#|include: false

%%sql inv <<
with plan(year, interest, principal, extra_payment, mortgage_balance, investment, investment_balance) as
(
    SELECT 0 AS year,
           NULL AS interest,
           NULL AS principal,
           NULL AS extra_payment,
           :principal AS mortgage_balance,
           NULL AS investment,
           0 AS investment_balance
    UNION
    SELECT p.year + 1 AS year,
           - p.mortgage_balance * :interest_rate AS interest,
           MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) AS principal,
           0 AS extra_payment,
           p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ +  
                           0 /*extra payment*/) AS mortgage_balance,
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END +
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate 
           ELSE 0 END AS investment,
           p.investment_balance*(1+:reference_rate) + 
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END +
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate 
           ELSE 0 END AS investment_balance
      FROM plan p
     WHERE p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ >= 0 
       AND MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ > 0 
)
SELECT year as year, 
       interest AS interest, 
       principal AS principal, 
       extra_payment AS extra_payment,
       -mortgage_balance AS mortgage_balance,
       investment AS investment,
       investment_balance AS investment_balance, 
       sum(principal+extra_payment)over(order by year)*(power(1+:appreciation, year)-1) AS appreciation
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable inv


In [14]:
#|echo: false

def plan_layout(df):
    df = df.DataFrame().set_index('year').reset_index(drop = True).rename_axis('Year', axis=1)
    df.loc['Total:'] = df.sum()
    df.loc['Total:',['mortgage_balance']]=df[['mortgage_balance']].max() #,'investment'
    df.loc['Total:',['appreciation']]=df[['appreciation']].values[-2:][0]
    df.loc['Total:',['investment_balance']]=df['investment_balance'].values[-2:][0]
    df['Net Worth'] = df[['principal','extra_payment']].cumsum().sum(axis=1) + df['investment_balance']
    df.loc['Total:','Net Worth'] = df.loc['Total:'][['principal','extra_payment','mortgage_balance','investment_balance']].sum()
    df['Net Worth (appr.)'] = df['Net Worth']  + df['appreciation']
    df_ret = df.copy()
    
    df.columns = pd.MultiIndex.from_tuples([('Mortgage', 'Interest'),
                                            ('Mortgage', 'Principal'),
                                            ('Mortgage', 'Extra Payment'),
                                            ('Mortgage', 'Loan Balance'),
                                            ('Investment', 'Payment'),
                                            ('Investment', 'Balance'),
                                            ('Appreciation', 'Balance'),
                                            ('Net Worth', 'w/o appr.'),
                                            ('Net Worth', 'with appr.')], names=['','Year'])
                                            
    vmin = -interest_rate*principal
    vmax = m_payment-interest_rate*principal
    display(df.style
                .format('{:.0f}', na_rep="")
                .bar(subset=pd.IndexSlice[df[df[('Mortgage', 'Interest')]<=0].index,('Mortgage', 'Interest')],
                     align='right', vmin=vmin, vmax=0, cmap="autumn", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
                .bar(subset=pd.IndexSlice[df[df[('Mortgage', 'Principal')]>=0].index,('Mortgage', 'Principal')],
                     align='left', vmin=0, vmax=m_payment, cmap="summer_r", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
                .bar(subset=[('Mortgage', 'Extra Payment')], align='left', vmin=0, vmax=lump_sum, cmap="summer_r", 
                     height=80, width=80, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                    )
                .bar(subset=[('Investment', 'Payment')], align='left', vmin=0, vmax=max(lump_sum, m_payment), cmap="summer_r", 
                     height=80, width=80, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                    )
                #.bar(subset=[('Mortgage', 'Interest')], align='right', vmin=vmin, vmax=0, cmap="autumn",
                #     height=80, width=60, props="width: 100px; border-right: 1px solid gray; border-left: 1px solid gray;"
                #    )
                #.bar(subset=[('Mortgage', 'Principal')], align='left', vmin=0, vmax=m_payment, cmap="RdYlGn", 
                #     height=80, width=60, props="width: 100px; border-right: 1px solid gray;"
                #    )
                .bar(subset=[('Mortgage', 'Loan Balance')], align='right', vmin=-2.5*principal, vmax=0, cmap="PuRd_r", 
                     height=80, width=100, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                    )
                .bar(subset=[('Appreciation', 'Balance')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                     height=80, width=100, props="width: 80px; border-right: 1px solid gray;"
                    )
                .bar(subset=[('Investment', 'Balance')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                     height=80, width=100, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                    )
                .bar(subset=[('Net Worth', 'w/o appr.')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                     height=80, width=100, props="width: 80px; border-right: 1px solid gray;"
                    )
                .bar(subset=[('Net Worth', 'with appr.')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                     height=80, width=100, props="width: 80px; border-right: 1px solid gray;"
                    )
                .set_table_styles({
                   ('Total:'): [{'selector': 'th', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'}]
                   }, overwrite=False, axis=1)
               .set_table_styles([{'selector': 'table', 'props': 'border-spacing: 2px'},
                                  {'selector': 'thead', 'props': 'border: 1px solid gray'}, 
                                  {'selector': 'th', 'props': 'text-align: center; padding: 4.5px;'},
                                  {'selector': 'th.col_heading', 'props': 'border: 1px solid gray'},
                                  {'selector': 'tbody', 'props': 'border: 1px solid gray'},
                                  {'selector': 'td', 'props': 'text-align: center; border-left: 1px solid gray; border-right: 1px solid gray;'}
                                 ], overwrite=False)
                .set_table_styles({
                        ('Investment', 'Balance'): [{'selector': 'th', 'props': 'border-right: 1px solid gray'},
                                   {'selector': 'td', 'props': 'border-right: 1px solid gray'}]
                        }, overwrite=False, axis=0)
    )
    
    return df_ret

inv_df = plan_layout(inv)

In [15]:
#|echo: false
duration = inv_df.index.values[-2]
md(f"The amortization duration is {duration} years.")

The amortization duration is 29 years.

During the time when the loan is paid off, the investments compounds. In the same time the value of the house raises as shown in the appreciation column.

# Extra loan payment
In the second scenario the lump sum is not invested but instead used as an **extra payment** to lower the loan.

In [16]:
#|include: false

%%sql so <<
with plan(year, interest, principal, extra_payment, mortgage_balance, investment, investment_balance) as
(
    SELECT 0 AS year,
           NULL AS interest,
           NULL AS principal,
           NULL AS extra_payment,
           :principal AS mortgage_balance,
           NULL AS investment,
           0 AS investment_balance
    UNION
    SELECT p.year + 1 AS year,
           - p.mortgage_balance * :interest_rate AS interest,
           MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) AS principal,
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 
                0
           END AS extra_payment,
           p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ +  
                           CASE WHEN :ls_year = p.year +1 THEN 
                                :lump_sum 
                           ELSE 
                                0
                           END /*extra payment*/) AS mortgage_balance,
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate 
           ELSE 0 END AS investment,       
           p.investment_balance*(1+:reference_rate) + 
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) +  
                           CASE WHEN :ls_year = p.year +1 THEN 
                                :lump_sum 
                           ELSE 
                                0
                           END) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate
           ELSE 0 END AS investment_balance 
      FROM plan p
     WHERE (p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) +  
                           CASE WHEN :ls_year = p.year +1 THEN 
                                :lump_sum 
                           ELSE 
                                0
                           END) /*mortgage_balance*/ >= 0 
       AND MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ > 0 
       ) OR p.year + 1 <= :duration
)
SELECT year, 
       interest AS interest, 
       principal AS principal, 
       extra_payment AS extra_payment,
       -mortgage_balance AS mortgage_balance,
       investment AS investment,
       investment_balance AS investment_balance,
       sum(principal+extra_payment)over(order by year)*(power(1+:appreciation, year)-1) AS appreciation
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable so


In [17]:
#|echo: false

so_df = plan_layout(so)

In [18]:
#|echo: false
duration = so_df[so_df['interest']==0].index.values[0]-1
md(f"The amortization duration is {duration} years.")

The amortization duration is 23 years.

The amortization plan shows that the loan is payed back earlier. So that the entire rate could afterwards be invested.

If we compare the *Total* row values of both scenarios we see that the interest rates paid to the bank is less, when the extra payment is done. Nonetheless both Net Worth values are bigger in the first scenario, when the money is invested.

So, :bust_in_silhouette: was right! But ...

# Comparing the strategies

Finally let's compare the Net Worh values of both scenarios by calculating their difference (```<>```).

In [19]:
#|echo: false

compare_df = inv_df[['Net Worth', 'Net Worth (appr.)']].merge(so_df[['Net Worth', 'Net Worth (appr.)']], left_index = True, right_index = True)
compare_df.columns = ['NW invest', 'NW invest (appr.)', 'NW extra pay', 'NW extra pay (appr.)']
compare_df['diff'] = compare_df['NW extra pay'] - compare_df['NW invest']
compare_df['diff (appr.)'] = compare_df['NW extra pay (appr.)'] - compare_df['NW invest (appr.)']

compare_df = compare_df[['NW invest', 'diff', 'NW extra pay', 'NW invest (appr.)', 'diff (appr.)', 'NW extra pay (appr.)']]


vmin=compare_df['diff (appr.)'].min()
vmax=compare_df['diff (appr.)'].max()

compare_df.columns = pd.MultiIndex.from_tuples([('Net Worth (w/o appreciation)', 'Investment'),
                                                ('Net Worth (w/o appreciation)', '<>'),
                                                ('Net Worth (w/o appreciation)', 'Extra Payment'),
                                                ('Net Worth (with appreciation)', 'Investment'),
                                                ('Net Worth (with appreciation)', '<>'),
                                                ('Net Worth (with appreciation)', 'Extra Payment')], names=['','Year'])


(compare_df.style
           .format('{:.0f}', na_rep="")
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Net Worth (w/o appreciation)', '<>')]>=0].index,('Net Worth (w/o appreciation)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="summer_r", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               )
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Net Worth (w/o appreciation)', '<>')]<=0].index,('Net Worth (w/o appreciation)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="autumn", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               ) 
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Net Worth (with appreciation)', '<>')]>=0].index,('Net Worth (with appreciation)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="summer_r", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               )
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Net Worth (with appreciation)', '<>')]<=0].index,('Net Worth (with appreciation)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="autumn", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               ) 
           .set_table_styles({
                   ('Total:'): [{'selector': 'th', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'}]
                   }, overwrite=False, axis=1)
           .set_table_styles([{'selector': 'table', 'props': 'border-spacing: 2px'},
                              {'selector': 'thead', 'props': 'border: 1px solid gray'}, 
                              {'selector': 'th', 'props': 'text-align: center; padding: 4.5px;'},
                              {'selector': 'th.col_heading', 'props': 'border: 1px solid gray'},
                              {'selector': 'tbody', 'props': 'border: 1px solid gray'},
                              {'selector': 'td', 'props': 'text-align: center; border-left: 1px solid gray; border-right: 1px solid gray'}
                             ], overwrite=False)
           .set_table_styles({
                   #('interest'): [{'selector': 'th', 'props': 'border-left: 1px solid gray'},
                   #           {'selector': 'td', 'props': 'border-left: 1px solid gray'}],
                   ('Net Worth (w/o appreciation)', 'investing'): [{'selector': 'th', 'props': 'border-left: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-left: 1px solid gray'}],
                   ('Net Worth (w/o appreciation)', 'extra payment'): [{'selector': 'th', 'props': 'border-right: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-right: 1px solid gray'}]
                   }, overwrite=False, axis=0)
)

As seen before, the Net Worth of the investment scenario is higher at the very end.
But what if we take appreciation into account and sold the house right after it's been payed of or even before. In this case the scenario with the extra payment can be superior (depending of the value of the reference rate).

By the way, in case the house looses value (negative appreciation) the extra payment and selling the house early can have a very negative effect.

# Conclusion
Even when the math seems easy it's always a good idea to open a spreadsheet and plot some charts to see what's going on and how different scenarios play out.

# Ressources
## Interactive notebook
- Play with the numbers: [![notebook]('../../../../assets/colab.svg?sanitize=true')](https://colab.research.google.com/github/joatom/blog/blob/master/assets/resources/notebooks/2022-11-13-repay-vs-invest.ipynb)
- Downloading the notebook: [![notebook]('../../../../assets/github.svg?sanitize=true')](https://github.com/joatom/blog/blob/master/assets/resources/notebooks/2022-11-13-repay-vs-invest.ipynb)

## Snippets
- [Recursive SQL](../../snippets/2022-11-20-recursive-sql/index.ipynb)
- [Styles in Pandas](../../snippets/2022-11-20-pandas-styles/index.ipynb)